In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!unzip gym-foo.zip
!pip install -e /content/gym-foo

In [ ]:
!pip install tf-agents

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

In [ ]:
import os, sys

import gym_foo
import tensorflow as tf
from tf_agents.environments import suite_gym

import imageio
from absl import logging
from tf_agents.environments import tf_py_environment
from tf_agents.environments.py_environment import PyEnvironment
from tf_agents.environments.tf_environment import TFEnvironment
from tf_agents.policies import tf_policy
from tf_agents.trajectories.policy_step import PolicyStep


In [ ]:

env_name = 'foo-v0'

def env_load_fn():
  py_env = suite_gym.load(env_name)
  py_env.reset()
  return py_env


In [ ]:

def create_video(py_environment: PyEnvironment, tf_environment: TFEnvironment, policy, num_episodes=1, video_filename='imageio.mp4'):
  logging.info("Generating video %s" % video_filename)
  with imageio.get_writer(video_filename, fps=1) as video:
    for episode in range(num_episodes):
      logging.info("Generating episode %d of %d" % (episode, num_episodes))

      time_step = tf_environment.reset()
      state = policy.get_initial_state(tf_environment.batch_size)

      video.append_data(py_environment.render())

      while not time_step.is_last():
        policy_step: PolicyStep = policy.action(time_step, state)
        state = policy_step.state
        act = policy_step.action

        time_step = tf_environment.step(act)

        img_bgr = py_environment.render()

        video.append_data(img_bgr)

  logging.info("Finished video %s" % video_filename)

In [ ]:
policy_dir = '/content/gdrive/My Drive/DATA691_RL_logs6/policy_saved_model/policy_000328000'

saved_policy = tf.compat.v2.saved_model.load(policy_dir)

In [ ]:
videos_dir = '/content/gdrive/My Drive/DATA691_RL_logs6'

video_file_name = os.path.join(videos_dir, "video_policy%d.mp4" % 180000_3)

In [ ]:

eval_py_env = env_load_fn()
eval_tf_env = tf_py_environment.TFPyEnvironment(eval_py_env)
create_video(eval_py_env, eval_tf_env, saved_policy, num_episodes=1, video_filename=video_file_name)